In [13]:
# import tensorflow and numpy
import tensorflow as tf
import numpy as np

In [20]:
# setup the parameters
# number of input values
vals = 3
# max answer, so basically the width of the frame
max_answer = 100
# number of different M's, biggest gradient will fit in frame
gradients = max_answer/(vals)+1
iterations = 5000
learning_rate = 0.5

# defining function to make training data
def training_data():
    
    M = 0
    rows = 1
    # array which we are using as our x values 
    # in equation of linear line, y = Mx
    # it includes 1 extra value as this will be used as our label
    x = np.arange(vals+1).astype(np.int32)
    # empty array to write our training data to
    y = np.array([])
    
    # loop so it cycles through every gradient
    for i in range(gradients):
        
        y = np.append(y, x*M).reshape(rows,vals+1)
            
        # increase number of rows to reshape it
        rows += 1
        
        # increase gradient by 1
        M+=1
     
    y = y.astype(np.int32)
    # return the training data
    # and number of lines to learn
    return(y,np.size(y,0))

# print the training data
print(training_data())

(array([[ 0,  0,  0,  0],
       [ 0,  1,  2,  3],
       [ 0,  2,  4,  6],
       [ 0,  3,  6,  9],
       [ 0,  4,  8, 12],
       [ 0,  5, 10, 15],
       [ 0,  6, 12, 18],
       [ 0,  7, 14, 21],
       [ 0,  8, 16, 24],
       [ 0,  9, 18, 27],
       [ 0, 10, 20, 30],
       [ 0, 11, 22, 33],
       [ 0, 12, 24, 36],
       [ 0, 13, 26, 39],
       [ 0, 14, 28, 42],
       [ 0, 15, 30, 45],
       [ 0, 16, 32, 48],
       [ 0, 17, 34, 51],
       [ 0, 18, 36, 54],
       [ 0, 19, 38, 57],
       [ 0, 20, 40, 60],
       [ 0, 21, 42, 63],
       [ 0, 22, 44, 66],
       [ 0, 23, 46, 69],
       [ 0, 24, 48, 72],
       [ 0, 25, 50, 75],
       [ 0, 26, 52, 78],
       [ 0, 27, 54, 81],
       [ 0, 28, 56, 84],
       [ 0, 29, 58, 87],
       [ 0, 30, 60, 90],
       [ 0, 31, 62, 93],
       [ 0, 32, 64, 96],
       [ 0, 33, 66, 99]], dtype=int32), 34)


In [21]:
# training_line_data = training data in numbers
# training lines = number of different lines
training_line_data,training_lines = training_data()

# the length is for when we convert the numbers into a binary array
# the array will be all zeros except one, which will be 1
# this will be the particle in this pont in time
# each one is like a frame in a video
# this value is the size of the largest M value multiplied by largest x values
length = (gradients-1)*vals-1

# the full length is the length of all the input frames stacked into one, 1d array
full_length = length*vals

# this function turns the data into the arrays explained above
def set_data():
    
    # there are two arrays, one for training data and one for labels
    # input_data  is the converted traiing data
    input_data = np.zeros([training_lines,vals,length])
    # the labels will be one-hot arrays
    labels = np.zeros([training_lines,1,gradients])
    
    # this sets the values specified in the training data to one
    for i in range(training_lines):
        # we need to set each individual input value
        for a in range(vals):
            # set the value to a 1
            input_data[i][a][training_line_data[i][a]] = 1
            
        # set the label value to a 1
        labels[i][0][training_line_data[i][vals-(vals-1)]] = 1
        
    # here, we reshape it tto the full length 1d array
    input_data = input_data.reshape(training_lines,1,full_length)
    
    # return the data and labels
    return(input_data,labels)    

# print converted data
#print(set_data())

In [22]:
# we define the weights, biases and inputs
# this will be input training data
x = tf.placeholder(tf.float32, [None, full_length])
# weights and biases
W = tf.Variable(tf.zeros([full_length, gradients]))
b = tf.Variable(tf.zeros([gradients]))
# function which gives the output of training data
y = tf.matmul(x, W) + b

# we will feed the labels in here
y_ = tf.placeholder(tf.float32, [None, gradients])

# configure the loss function, using cross entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# define the optimizer
# AdagradOptimizer works much better than GradientDescentOptimizer
#train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)
train_step = tf.train.AdagradOptimizer(learning_rate).minimize(cross_entropy)


In [23]:
# I am using a GPU
# this line limits memory usage of the GPU to 0.4 when session is created
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)

# create interactive session using the GPU line for above
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options)) 

# initialize variables
init = tf.global_variables_initializer()
sess.run(init)

# now we run the learning loop
for _ in range(iterations):

    # set training data and labels
    x_data, y_data = set_data()
    # only use one lines data
    # chooses line data by evenly spreading data over iterations
    x_data = x_data[np.round(_//(iterations/(training_lines-0.)), 0).astype(np.int32)]
    y_data = y_data[np.round(_//(iterations/(training_lines-0.)), 0).astype(np.int32)]
    
    # run the training optimizer
    sess.run(train_step, feed_dict={x: x_data, y_: y_data})

    # print steps and error 10 times in total
    if _ % (iterations/10) == 0:
        print '\nstep', _, 'out of', iterations
        print 'line', np.round(_//(iterations/(training_lines-0.)), 0).astype(np.int32), 'out of', training_lines
        print 'error =', sess.run(cross_entropy, feed_dict={x: x_data, y_: y_data})


step 0 out of 5000
line 0 out of 34
error = 1.62819

step 500 out of 5000
line 3 out of 34
error = 0.0130336

step 1000 out of 5000
line 6 out of 34
error = 0.00677739

step 1500 out of 5000
line 10 out of 34
error = 0.0307595

step 2000 out of 5000
line 13 out of 34
error = 0.0100413

step 2500 out of 5000
line 16 out of 34
error = 0.00615902

step 3000 out of 5000
line 20 out of 34
error = 0.016889

step 3500 out of 5000
line 23 out of 34
error = 0.00834987

step 4000 out of 5000
line 27 out of 34
error = 0.0387216

step 4500 out of 5000
line 30 out of 34
error = 0.011867


In [28]:
# initialize array to record test values
test_line = np.array([])

print 'input values, the max gradient is', gradients-1
# loop to ask user for input values
for i in range(vals):
    print '\nvalue',i+1,'?'
    val = int(input())
    test_line = np.append(test_line, val)
    
test_line = test_line.astype(np.int32)
# record the last value of the points
# this is needed because we have only worked out the gradient not the bias
last_val = test_line[-1]
# show line that user has inputed
print(test_line)

# function to convert data into training data format
def test_model():
    # first value of the points to take away the b later
    global test_line
    first_value = test_line[0]
    
    # take away the first value so its only left with the difference between
    # the points and predicts y = Mx , not y = Mx+ b
    test_line = np.subtract(test_line,first_value)


    input_array = np.array([])
    # for each input value, write to the input array
    # with a 1 in the position
    for a in range(vals):
        test_array = np.zeros([1,length])
        test_array[0][test_line[a]] = 1
        # we write to array and shape so new line for each test value
        input_array = np.append(input_array, test_array).reshape([1,(a+1)*length])
        
    # return array
    return(input_array)   

# print array
#test_model()

input values, the max gradient is 33

value 1 ?
33

value 2 ?
54

value 3 ?
75
[33 54 75]


In [29]:
# run the function to get output
# using new weights and biases
input_array = test_model()
probs = sess.run(y, feed_dict={x:input_array})
# now we have found the predicted gradient,
# we add the last value to get the next value
learnt_ans = np.argmax(probs)+last_val
# "true" answer, should work with all linear lines
# this numerically works out hte gradient by findingthe difference,
# then also adds the last value
answer = (test_line[1]-test_line[0])+last_val

if answer == learnt_ans:
    print 'Correct'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', learnt_ans
    print 'Probaility of Gradient', np.argmax(probs), '=', probs[0][learnt_ans-last_val]
else:
    print 'Wrong'
    
    # print learnt answer and it probability
    print '\nLearnt answer =', learnt_ans
    print 'Probaility of Gradient', np.argmax(probs), '=', probs[0][learnt_ans-last_val]
    
    # print true answer and its probabilities
    print '\nNumerical answer =', answer
    print 'Probability of Gradient', test_line[1]-test_line[0], '=', probs[0][answer-last_val]
    

# print the probabilities
print '\n\n', probs

Correct

Learnt answer = 96
Probaility of Gradient 21 = 3.05971


[[-3.66655993 -3.65550804 -3.65389061 -3.64971638 -3.64564228 -3.64302254
  -3.64269495 -3.64526844 -3.65133047 -3.66149068 -3.6764698  -3.69711781
  -3.72443533 -3.75961161 -3.80402374 -3.85921764 -3.92737961 -4.00873899
  -4.10605049 -4.21855736 -4.33970737  3.05971193 -1.93413854 -1.82842946
  -1.71321583 -1.58714485 -1.4485817  -1.29553092 -1.12550795 -0.93533027
  -0.7207495  -0.4757531  -0.19094923  0.15091932]]
